<a href="https://colab.research.google.com/github/Chaitra-B-V/EDA-ApacheBeam/blob/main/Apache_Beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using Composite Transform to filter the census data withrespect to employee type and retrieving the number of records with filtered type( private and govt employee)

In [ ]:
!pip install apache_beam

In [13]:
import apache_beam as beam
#composite transform
class CustomTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    a = ( 
        input_coll
                       | 'Group and sum' >> beam.CombinePerKey(sum)
                       | 'count filter accounts' >> beam.Filter(filter_on_count)
                       | 'Regular accounts employee' >> beam.Map(format_output)
              
    )
    return a
#to split the file with delimiter ','
def SplitRow(element):
    return element.split(',')
  
#count the no of records  
def filter_on_count(element):
  name, count = element
  if count > 1:
    return element

#output format  
def format_output(element):
  name, count = element
  #return ', '.join((name.encode('ascii'),str(count),'Regular employee'))
  return ', '.join((name,str(count),'employee type'))
#creating beam object
p = beam.Pipeline()

#read input file
input_collection = ( 
                      p 
                      | "Read from text file" >> beam.io.ReadFromText('/content/Apache_Beam.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )
#count for private employees
private_count = (
                      input_collection
                      | 'Get all Private Employees' >> beam.Filter(lambda record: record[1] == ' Private')
                      | 'Pair each employee with 1' >> beam.Map(lambda record: (" Private, " +record[1], 1))
                      | 'composite private employee' >> CustomTransform()
                      | 'Write results for private employee' >> beam.io.WriteToText('us.txt')
                 )

#count for state employees
state_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[1] == ' State-gov')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: (" State-gov, " +record[1], 1))
                | 'composite state gov' >> CustomTransform()
                | 'Write results for state employee' >> beam.io.WriteToText('germany.txt')
           ) 
p.run()